In [1]:
!unzip 8503d976798f11ea.zip

Archive:  8503d976798f11ea.zip
   creating: Dataset/
  inflating: Dataset/Train.csv       
  inflating: Dataset/sample_submission.csv  
  inflating: Dataset/Test.csv        


In [2]:
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble

import pandas, xgboost, numpy, textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers 

import pandas as pd

Using TensorFlow backend.


In [3]:
# read data
train_df = pd.read_csv("./Dataset/Train.csv")
test_df = pd.read_csv("./Dataset/Test.csv")
print(train_df.columns)

drop_list = ['ID', 'date', 'user', 'topic']
train_df.drop(drop_list, axis=1, inplace=True)
test_df.drop(drop_list, axis=1, inplace=True)

train_df["full_comment"] = train_df["comment"] +train_df["parent_comment"]
test_df["full_comment"] = test_df["comment"] +test_df["parent_comment"]

Index(['ID', 'comment', 'date', 'down', 'parent_comment', 'score', 'top',
       'topic', 'user', 'label'],
      dtype='object')


In [4]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [5]:
from nltk.corpus import wordnet

def get_wordnet_pos(pos_tag):
    if pos_tag.startswith('J'):
        return wordnet.ADJ
    elif pos_tag.startswith('V'):
        return wordnet.VERB
    elif pos_tag.startswith('N'):
        return wordnet.NOUN
    elif pos_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [6]:
import string
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.tokenize import WhitespaceTokenizer
from nltk.stem import WordNetLemmatizer

In [7]:
def clean_text(text):
    # lower text
    text = text.lower()
    # tokenize text and remove puncutation
    text = [word.strip(string.punctuation) for word in text.split(" ")]
    # remove words that contain numbers
    text = [word for word in text if not any(c.isdigit() for c in word)]
    # remove stop words
    stop = stopwords.words('english')
    text = [x for x in text if x not in stop]
    # remove empty tokens
    text = [t for t in text if len(t) > 0]
    # pos tag text
    pos_tags = pos_tag(text)
    # lemmatize text
    text = [WordNetLemmatizer().lemmatize(t[0], get_wordnet_pos(t[1])) for t in pos_tags]
    # remove words with only one letter
    text = [t for t in text if len(t) > 1]
    # join all
    text = " ".join(text)
    return(text)

In [8]:
train_df["full_comment_clean"] = train_df["full_comment"].apply(clean_text)
test_df["full_comment_clean"] = test_df["full_comment"].apply(clean_text)
print(train_df.head())

df = train_df["full_comment_clean"] + test_df["full_comment_clean"]
df = df.fillna(" ")


                                             comment  ...                                 full_comment_clean
0  Well, let's be honest here, they don't actuall...  ...  well let's honest actually seem much moderate ...
1  Well, I didn't need evidence to believe in com...  ...  well need evidence believe communism.you need ...
2              Who does an "official promo" in 360p?  ...                  official promo bmw official promo
3                           Grotto koth was the best  ...  grotto koth bestnot really memorable lol want ...
4                                   Neal's back baby  ...         neal's back babyjames neal hit zach parise

[5 rows x 8 columns]


In [9]:
tokenizer = text.Tokenizer()
tokenizer.fit_on_texts(df)
print(tokenizer.word_index)
train_x = tokenizer.texts_to_sequences(train_df["full_comment_clean"])
test_x = tokenizer.texts_to_sequences(test_df["full_comment_clean"])

train_x = sequence.pad_sequences(train_x, maxlen=100)
test_x = sequence.pad_sequences(test_x, maxlen=100)
train_y = train_df.label.values


{'get': 1, 'like': 2, 'people': 3, 'would': 4, 'go': 5, 'make': 6, 'one': 7, 'think': 8, 'say': 9, "i'm": 10, 'know': 11, 'game': 12, 'time': 13, 'good': 14, 'want': 15, 'well': 16, 'see': 17, 'use': 18, 'i': 19, 'thing': 20, 'really': 21, 'even': 22, 'take': 23, 'need': 24, 'right': 25, 'look': 26, 'play': 27, "that's": 28, 'guy': 29, 'way': 30, 'could': 31, 'year': 32, 'work': 33, 'fuck': 34, 'much': 35, 'new': 36, 'come': 37, 'still': 38, 'give': 39, 'try': 40, "can't": 41, 'never': 42, 'mean': 43, 'bad': 44, 'also': 45, 'something': 46, 'sure': 47, 'back': 48, 'first': 49, 'actually': 50, 'shit': 51, 'find': 52, 'yeah': 53, 'tell': 54, 'day': 55, 'start': 56, 'someone': 57, 'man': 58, 'post': 59, 'team': 60, 'pretty': 61, "he's": 62, 'every': 63, 'call': 64, 'best': 65, 'great': 66, 'lot': 67, "i've": 68, 'love': 69, 'point': 70, 'feel': 71, 'kill': 72, 'two': 73, 'around': 74, 'show': 75, 'probably': 76, 'everyone': 77, 'player': 78, 'life': 79, 'ever': 80, 'put': 81, 'anything': 

In [10]:
from sklearn.model_selection import train_test_split

x_train, x_valid, y_train, y_valid = train_test_split(train_x, train_y, test_size = 0.20, random_state = 42)

from __future__ import print_function

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding, LSTM

In [ ]:
print('Build model...')
model = Sequential()
model.add(Embedding(40000, 128, input_length=100))
model.add(LSTM(64, return_sequences=True))
model.add(LSTM(64))
model.add(Dense(1, activation="sigmoid"))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
              
model.fit(x_train, y_train,
          batch_size=32,
          epochs=2,
          validation_data=(x_valid, y_valid))


Build model...


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 12000 samples, validate on 3000 samples
Epoch 1/2
 2496/12000 [=====>........................] - ETA: 1:54 - loss: 0.6911 - accuracy: 0.5276

In [ ]:
import numpy as np

y_test_pred = np.round(model.predict(test_x)).flatten()

test_df = pd.read_csv("./Dataset/Test.csv")
print(test_df.ID.values)
result_df = pd.DataFrame({'ID':test_df.ID.values, 'label':y_test_pred})
result_df.to_csv("icertis_submission.csv", index=False)

from sklearn.metrics import f1_score, accuracy_score
import numpy as np
y_pred = np.round(model.predict(x_valid))
print(f1_score(y_valid, y_pred, average='weighted'))
print(accuracy_score(y_valid, y_pred))